<a href="https://colab.research.google.com/github/Yankee1231/Microprocessor_practice/blob/main/Microprocessor_Week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 마이크로 프로세서 5주차 코드
-----
### Port Acess Code
```cpp
//캐릭터 LCD 포트제어방식(4비트모드)
#include <avr/io.h>
#include <avr/delay.h>
typedef unsigned char byte;

void lcd_iw(byte inst) //LCD명령출력함수.
{
  PORTG = 0x04;       //LCD를 명령쓰기 모드로 설정
  PORTD = inst&0xf0;  //명령어 상위 4비트 전송
  PORTG = 0xfb;       //LCD Disable (E=0)
  PORTG = 0x04;       
  PORTD = inst<<4;    //명령어 하위 4비트 전송
  PORTG = 0xfb;       //LCD Disable
  _delay_ms(2);       //명령어 사이 휴식
}
void lcd_dw(byte dw)  //LCD 데이터 출력 함수
{
  PORTG = 0x05;       //LCD를 데이터 쓰기 모드로 설정
  PORTD = dw&0xf0;    //데이터 상위 4비트 전송
  PORTG = 0xfb;       //LCD Disable (E=0)
  PORTG = 0x05;
  PORTD = dw<<4;      //데이터하위4비트 전송
  PORTG = 0xfb;
  _delay_ms(2);
}

void init_lcd(void) //LCD초기화
{
  lcd_iw(0x20);
  lcd_iw(0x28); //FunctionSet
  lcd_iw(0x06); //EntryModeSet
  lcd_iw(0x0c); //DisplayOnOffControl
  lcd_iw(0x01); //DisplayClear
}

void lcd_str(char *str) // 문자열 출력함수
{
  byte i;
  while(*str){
    lcd_dw(*str++);
    for(i=0;i<10;i++)
      _delay_ms(10); // 시간 지연
  }
}

int main(void)
{
  byte i;
  byte *str1 = "<masitech.co.kr>";
  byte *str2 = "-Kit is Good!";
  DDRD = 0xf0;
  DDRG = 0x07;
  init_lcd();
  do {
    lcd_str(str1);
    lcd_iw(0xc0);
    lcd_dw('A'); for(i=0; i<10; i++) _delay_ms(10);
    lcd_dw(''); for(i=0; i<10; i++) _delay_ms(10);
    lcd_dw(0xe2); for(i=0; i<10; i++) _delay_ms(10);
    lcd_str(str2); for(i=0; i<10; i++) _delay_ms(10);
    lcd_iw(0x01); for(i=0; i<10; i++) _delay_ms(10);
  }while(1);
```

### Code 1: Print 'A' on LCD
```cpp
#include <avr/io.h>

#define LCD_EN (*(volatile unsigend char*)0x8000)
#define LCD_Write(*(volatile unsigend char*)0x8002)

// 함수선언부 - Delay 함수
void delay_us(unsigned char time_us);
void delay_ms(unsigned char time_ms);

// 함수선언부 - LCD 초기화 함수
void FunctionSet();
void DisplayOnOffControl();
void DisplayClear();
void EntryModeSet();

// 함수선언부 - 글자표시함수
void DisplayChar(unsigned char c);

int main(void)
{
  // SFR초기화
  MCUCR = 0x80; //I/O포트를 특수기능 모드로 설정 , 외부 메모리를 사용
  XMCRA = 0x40; //외부메모리 섹터 설정: 0x8000~0xFFFF;
  XMCRB = 0x80; //외부메모리 섹터 설정: 주소 ativation AN0~7

  //LCD 초기화
  FunctionSet();
  DisplayOnOffControl();
  DisplayClear();
  EntryModeSet();

  //LCD에 테스트 문자열 표시
  DisplayChar('A');

  while(1)
  {
    // Main Loop
  }
}

void delay_us(unsinged char time_us)
{
  unsigned char i;
  for(i=0;i<time_us;i++)
  {
    asm volatile("PUSH R0");
    asm volatile("POP R0");
    asm volatile("PUSH R0");
    asm volatile("POP R0");
    ams volatile("PUSH R0");
    ams volatile("POP R0");
  }
}

void delay_ms(unsigned int time_ms)
{
  unsigned int i;
  for(i=0;i<time_ms;i++)
  {
    delay_us(250);
    delay_us(250);
    delay_us(250);
    delay_us(250);
  }
}

void FunctionSet()
{
  delay_ms(50);
  LCD_EN = 0x38; // 8bits, 2lines, 5x7dots
}

void DisplayOnOffControl()
{
  delay_us(40);
  LCD_EN = 0x0c;
}

void DisplayClear()
{
  delay_us(40);
  LCD_EN = 0x01;
}

void EntryMddeSet()
{
  delay_ms(2);
  LCD_EN = 0x06;
  delay_us(40);
}

void DisplayChar(unsigned char c);
{
  LCD_Write = c;
  delay_us(50);
}
```


### Code 2: Print "Hello World" on LCD
```cpp
#include <avr/io.h>

// LCD 모듈로 접근하기 위한 외부 메모리 주소 선언
#define LCD_EN (*(volatile unsigned char*)0x8000)
#define LCD_Write(*(volatile unsigned char*)0x8002)

#define LINE1 0x00 // LCD Display line 1
#define LINE2 0x40 // LCD Display line 2

// 함수 선언부 - Delay 함수
void delay_us(unsigned char time_us);
void delay_ms(unsigned char time_ms);

// 함수 선언부 - LCD 초기화 함수
void FunctionSet();
void LCDOnOffControl();
void DisplayClear();
void EntryModeSet();

// 함수 선언부 - LCD 글자표시 함수
void DisplayChar(unsigned char c);
void DisplayString(unsigned char *s);
void SetLCDAddress(unsigned char mode);

// LCD에 표시할 문자열
char*s = "Hello  ";
char*s2 = "World  ";

int main(void)
{
  // SRF
  MCUCR = 0x80; //I/O포트를 특수기능 모드로 설정
  XMCRA = 0x40; // 외부메모리 섹터 설정: 0x8000~0xFFFF
  XMCRB = 0x80; // 외부메모리 섹터 설정: 버스 Activation AD0~7

  // LCD 초기화 함수
  FunctionSet();
  DispalyOnOffControl();
  DisplayClear();
  EntryModeSet();

  // LCD 첫줄에 Hello 표시
  SetLCDAddress(LINE1);
  DisplayString(s);

  // LCD 둘째줄에 World 표시
  SetLCDAddress(LINE2);
  DisplayString(s2);

  while(1)
  {

  }
}

void delay_us(unsigned char time_us)
{
  unsigned char i;
  for(i=0;i<time_us;i++)
  {
    asm volatile("PUSH RO");
    asm volatile("POP R0");
    asm volatile("PUSH RO");
    asm volatile("POP R0");
    asm volatile("PUSH RO");
    asm volatile("POP R0");
  }
}

void delay_ms(unsigned int time_ms)
{
  unsigned int i;
  for(i=0;i<i=time_ms;i++)
  {
    delay_us(250);
    delay_us(250);
    delay_us(250);
    delay_us(250);
  }
}

void FunctionSet()
{
  delay_ms(50);
  LCD_EN = 0x38; // 8bits, 2lines, 5x7 dots
}

void DisplayOnOffControl()
{
  delay_us(40);
  LCD_EN = 0x0c; // Display on/off ON
}

void DisplayClear()
{
  delay_us(40);
  LCD_EN = 0x01; // Display clear
}

void EntryModeSet()
{
  delay_ms(2);
  LCD_EN = 0x06; // Increment Mode
}

void DisplayChar(unsigned char c)
{
  LCD_Write = c;
  delay_us(50);
}

void DisplayString(unsigned char *s)
{
  //문자열 출력함수, 주어진 문자열의 길이만큼 반복 출력한다.
  unsigned char i,j;
  i = 0;
  j = 8-1; //8글자 LCD이므로 0~7
  for(i=0;i<=j;i++)
  {
    DisplayChar(s[i]);
  }
}

void SetLCDAddress(unsigned char mode)
{
  //어드레스 세팅을 위해서는 MSB는 1이 되어야 함.
  //사용자가 원한 주소를 받아 MSB를 1로
  LCD_EN = 0x80|mode;
  delay_us(50);
}
```